In [7]:
import math
#import numpy
import autograd.numpy as np
from autograd import elementwise_grad
from scipy.special import comb
import itertools

In [72]:
state = np.array([1.0, 2, 3, 4, 5, 6, 7, 8, 9])
numcards = 8
ownCards = np.array([1.0, 0, 0, 0, 0, 0, 0, 0, 0])
factor = 1

calls = np.array([[-1, -1, -1, -1, -1],
                  [ 0, -1, -1, -1, -1],
                  [ 1, -1,  2, -1, -1],
                  [ 3, -1,  4, -1, -1],
                  [ 5, -1, -1,  6, -1]])

def findCall(num):
    for i in range(calls.size):
        for j in range(calls[0].size):
            if calls[i][j] == num:
                return [i, j]

In [9]:
def oneCardProbabilities(state):
    total = np.sum(state)
    oneCardProb = (state+1)/(total+1)
    return oneCardProb

In [65]:
def twotypeCall(state, numcards, countOne, countTwo, typeOne, typeTwo):
    return twotype(state, numcards, countOne, countTwo)[typeOne][typeTwo]

def twotype(state, numcards, countOne, countTwo):
    prob = oneCardProbabilities(state)
    #probPairs = np.array(np.meshgrid(prob,prob)) # so that it has shape (2,13,13)
    probPairs = np.transpose(np.reshape(np.array(list(map(np.array,itertools.product(prob,prob)))),(9,9,2)),(2,0,1))
    
             #print(probPairs.shape)
#     probabilities = np.zeroes([len(state), len(state)])
#     for i in range(len(state)):
#         for j in range(len(state)):
#             twopairprob(probPairs[i, j], numcards)
    
    return twotypeprob(probPairs, numcards, countOne, countTwo)

def twotypeprob(p, numcards, countCardOne, countCardTwo):
    prob = 0
    for i in range(countCardOne, 5):
        for j in range(countCardTwo, 5):
            prob = prob + comb(numcards, i) * comb(numcards-i, j) * (p[0])**i*(p[1])**j*(1-(p[0]+p[1]))**(numcards-(i+j))
    return prob
            

def onetypeCall(state, numcards, count, cardType):
    return onetype(state, numcards, count)[cardType]
    
def onetype(state, numcards, count):
    prob = oneCardProbabilities(state)
    return onetypeprob(prob, numcards, count)
    
def onetypeprob(p, numcards, countCard):
    prob = 0
    for i in range(countCard, 5):
        prob = prob + comb(numcards, i)* p**i * (1-p)**(numcards-i)
    return prob

In [34]:
pgrad = elementwise_grad(twotypeCall)

In [35]:
pgrad(state, numcards, 2, 2, 0, 4)

array([ 0.00696966, -0.00060829, -0.00060829, -0.00060829,  0.00152857,
       -0.00060829, -0.00060829, -0.00060829, -0.00060829])

In [36]:
pograd = elementwise_grad(onetypeCall)

In [38]:
pograd(state, numcards, 2, 4)

array([-0.00429789, -0.00429789, -0.00429789, -0.00429789,  0.02865249,
       -0.00429789, -0.00429789, -0.00429789, -0.00429789])

In [15]:
def updateParameters(call, state, factor, numcards):
    if(call[1] == 0):
        grad = pograd(state, numcards, call[0], call[2])
    else:
        grad= pgrad(state, numcards, call[0], call[1], call[2], call[3])
    
    return state + factor*grad
    

In [16]:
state

array([1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [17]:
state = updateParameters(np.array([3, 2, 0, 4]), state, factor, numcards)


In [18]:
state

array([1.00058538, 1.99995808, 2.99995808, 3.99995808, 5.00007037,
       5.99995808, 6.99995808, 7.99995808, 8.99995808])

In [40]:
pgrad(state, numcards, 3, 2, 0, 4)

array([ 6.78143521e-04, -5.04149914e-05, -5.04149914e-05, -5.04149914e-05,
        9.31773844e-05, -5.04149914e-05, -5.04149914e-05, -5.04149914e-05,
       -5.04149914e-05])

In [84]:
def makeCall(call, state, numcards, calls):  #not quite
    allCallProb = [onetype(state, numcards, 1), onetype(state, numcards, 2), twotype(state, numcards, 2, 2), onetype(state, numcards, 3), twotype(state, numcards, 3, 2), onetype(state, numcards, 4), twotype(state, numcards, 4, 3)]
    ind = calls[call[0]][call[1]]
    bestCall = [-1, -1, -1, -1, 0]
    for i in range(ind+1, len(allCallProb)):  #call strength
        for j in range(allCallProb[i].shape[0]):  #level within call
            if len(allCallProb[i].shape) > 1:
                for k in range(allCallProb[i][j].shape[0]):
                    prob = allCallProb[i][j][k]
                    if(prob > bestCall[4]):
                        call = findCall(i)
                        bestCall = [call[0], call[1], j, k, prob]
                    
            else:
                
                prob = allCallProb[i][j]
                if(prob > bestCall[4]):
                    call = findCall(i)
                    bestCall = [call[0], call[1], j, -1, prob]
    return bestCall
    

In [88]:
def makeDecision(call, state, numcards, calls):
    #somewhat unnecessary
    allCallProb = [onetype(state, numcards, 1), onetype(state, numcards, 2), twotype(state, numcards, 2, 2), onetype(state, numcards, 3), twotype(state, numcards, 3, 2), onetype(state, numcards, 4), twotype(state, numcards, 4, 3)]
    
    ownCall = makeCall(call, state, numcards, calls)
    oppCallProb = 0
    if(call[1] == 0):
        oppCallProb = allCallProb[calls[call[0]][call[1]]][call[2]]
    else:
        oppCallProb = allCallProb[calls[call[0]][call[1]]][call[2]][call[3]]
    
    if(ownCall[4] > 1-oppCallProb):
        return 0
    else:
        return 1

In [22]:
state

array([1.00058538, 1.99995808, 2.99995808, 3.99995808, 5.00007037,
       5.99995808, 6.99995808, 7.99995808, 8.99995808])

In [29]:
x = np.array([1,2])

In [47]:
state

array([1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [80]:
makeCall(np.array([3, 2, 2, 1]), state, numcards, calls)

[4, 0, 8, -1, 0.058646944404034874]

In [89]:
makeDecision(np.array([3,2,2,1]), state, numcards, calls)

1

In [83]:
allCallProb

[array([0.29924923, 0.41692205, 0.5167949 , 0.60106232, 0.67159926,
        0.72998715, 0.77754278, 0.81534928, 0.84428811]),
 array([0.0444336 , 0.09151438, 0.14880801, 0.21247884, 0.27931713,
        0.3466529 , 0.41228246, 0.47440618, 0.5315759 ]),
 array([[0.00118353, 0.00250499, 0.00418648, 0.00614532, 0.00830759,
         0.0106074 , 0.01298629, 0.0153925 , 0.01778043],
        [0.00250499, 0.00529698, 0.00884427, 0.01296992, 0.01751608,
         0.02234237, 0.0273244 , 0.03235237, 0.03732973],
        [0.00418648, 0.00884427, 0.01475284, 0.02161332, 0.02915955,
         0.03715532, 0.04539177, 0.05368492, 0.06187346],
        [0.00614532, 0.01296992, 0.02161332, 0.03163207, 0.04263204,
         0.05426414, 0.06622039, 0.07823015, 0.0900567 ],
        [0.00830759, 0.01751608, 0.02915955, 0.04263204, 0.05739603,
         0.07297647, 0.08895517, 0.10496557, 0.12068781],
        [0.0106074 , 0.02234237, 0.03715532, 0.05426414, 0.07297647,
         0.09268174, 0.11284374, 0.13299358,

In [64]:
onetypeprob(prob, numcards, 2)

array([0.0442242 , 0.09054745, 0.14602655, 0.20631217, 0.26773255,
       0.32725807, 0.38246095, 0.43147109, 0.47292896])

In [61]:
prob = oneCardProbabilities(state)

In [62]:
prob

array([0.04347826, 0.06521739, 0.08695652, 0.10869565, 0.13043478,
       0.15217391, 0.17391304, 0.19565217, 0.2173913 ])